In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Working directory
%cd /content/drive/MyDrive/TechLabs/Project

# Import libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display


Mounted at /content/drive
/content/drive/MyDrive/TechLabs/Project


**Clean and prepare the data**

In [ ]:
# File paths
rent_file = 'data/BerlinPrices/AHS-Index-RENT-PLZ-2024-long.csv'
purch_file = 'data/BerlinPrices/AHS-Index-PURCH-PLZ-2024-long.csv'

# Datasets Rent and Purchase Prices (Germany)
rent_data = pd.read_csv(rent_file)
purch_data = pd.read_csv(purch_file)

print("Rental Data:")
display(rent_data.head())

print("\nPurchase Data:")
display(purch_data.head())


Rental Data:


,postcode_id,year,Obs,Radius,Obs_own,Radius_own,Effect_nown,lprice_qm,lprice_qm_se,price_qm,price_qm_se,target_x,target_y,target_id
0,1067,2007,791936,10,172214,2.5,-0.038581,1.751763,0.035531,5.867701,0.208483,830654.428259,5.666966e+06,1067
1,1067,2008,791936,10,172214,2.5,-0.038581,1.764310,0.028542,5.941785,0.169590,830654.428259,5.666966e+06,1067
2,1067,2009,791936,10,172214,2.5,-0.038581,1.769906,0.029300,5.975130,0.175072,830654.428259,5.666966e+06,1067
3,1067,2010,791936,10,172214,2.5,-0.038581,1.805676,0.034665,6.192730,0.214672,830654.428259,5.666966e+06,1067
4,1067,2011,791936,10,172214,2.5,-0.038581,1.865653,0.019675,6.575513,0.129370,830654.428259,5.666966e+06,1067



Purchase Data:


,postcode_id,year,Obs,Radius,Obs_own,Radius_own,Effect_nown,lprice_qm,lprice_qm_se,price_qm,price_qm_se,target_x,target_y,target_id
0,1067,2007,213267,10,30937,2.5,-0.016235,7.356732,0.086941,1683.2145,146.340160,830654.428259,5.666966e+06,1067
1,1067,2008,213267,10,30937,2.5,-0.016235,7.306041,0.080064,1600.0168,128.103640,830654.428259,5.666966e+06,1067
2,1067,2009,213267,10,30937,2.5,-0.016235,7.223010,0.122046,1472.5322,179.716200,830654.428259,5.666966e+06,1067
3,1067,2010,213267,10,30937,2.5,-0.016235,7.335990,0.019614,1648.6609,32.337154,830654.428259,5.666966e+06,1067
4,1067,2011,213267,10,30937,2.5,-0.016235,7.523266,0.037715,1988.2200,74.985008,830654.428259,5.666966e+06,1067


In [ ]:
# List of Berlin postal codes
berlin_postcodes = [
    '10115', '10559', '13355', '10117', '10623', '13357', '10119', '10785', '13359',
    '10178', '10787', '13405', '10179', '10963', '13407', '10435', '10969', '13409',
    '10551', '13347', '10553', '13349', '10555', '13351', '10557', '13353', '10967',
    '10243', '10245', '10997', '10247', '10999', '10249', '12045', '10367', '10961',
    '10965', '13053', '13187', '13086', '13189', '13088', '10405', '13089', '10407',
    '13125', '10409', '13127', '13129', '10437', '13156', '10439', '13158', '13051',
    '13159', '10711', '14193', '10585', '10713', '13597', '14195', '10587', '10715',
    '13627', '14197', '10589', '10717', '13629', '14199', '10719', '14050', '10625',
    '10777', '14052', '10627', '10779', '14053', '10629', '14055', '10707', '10789',
    '14057', '10709', '10825', '14059', '13581', '13583', '13585', '13587', '14089',
    '13589', '13591', '13593', '13595', '13599', '12157', '12247', '12161', '12249',
    '12163', '12277', '12165', '12279', '12167', '14109', '12169', '14129', '12203',
    '14163', '12205', '14165', '12207', '14167', '12209', '14169', '10829', '10781',
    '12099', '10783', '12101', '12103', '12105', '12107', '12305', '10823', '12109',
    '12307', '12309', '10827', '12159', '12347', '12059', '12359', '12043', '12047',
    '12049', '12349', '12051', '12351', '12053', '12353', '12055', '12355', '12057',
    '12357', '12435', '12557', '12437', '12559', '12439', '12587', '12459', '12589',
    '12487', '12623', '12489', '12524', '12526', '12527', '12555', '12687', '12619',
    '12689', '12621', '12627', '12629', '12679', '12681', '12683', '12685', '10315',
    '13057', '10317', '13059', '10318', '10319', '10365', '10369', '13055', '13403',
    '13503', '13505', '13507', '13509', '13435', '13437', '13439', '13465', '13467',
    '13469'
]

# Convert postal codes to string format
rent_data['postcode_id'] = rent_data['postcode_id'].astype(str)
purch_data['postcode_id'] = purch_data['postcode_id'].astype(str)

# Filter data for Berlin postal codes
berlin_rent_data = rent_data[rent_data['postcode_id'].isin(berlin_postcodes)]
berlin_purch_data = purch_data[purch_data['postcode_id'].isin(berlin_postcodes)]

# Results
print("Filtered Berlin Rental Data:")
display(berlin_rent_data.head())

print("\nFiltered Berlin Purchase Data:")
display(berlin_purch_data.head())


Filtered Berlin Rental Data:


,postcode_id,year,Obs,Radius,Obs_own,Radius_own,Effect_nown,lprice_qm,lprice_qm_se,price_qm,price_qm_se,target_x,target_y,target_id
11458,10115,2007,1330366,10,193637,2.5,0.005153,2.028197,0.042033,7.859726,0.330365,797406.274895,5.829286e+06,10115
11459,10115,2008,1330366,10,193637,2.5,0.005153,2.089856,0.055726,8.359600,0.465848,797406.274895,5.829286e+06,10115
11460,10115,2009,1330366,10,193637,2.5,0.005153,2.127689,0.038823,8.681931,0.337059,797406.274895,5.829286e+06,10115
11461,10115,2010,1330366,10,193637,2.5,0.005153,2.206238,0.031469,9.391384,0.295536,797406.274895,5.829286e+06,10115
11462,10115,2011,1330366,10,193637,2.5,0.005153,2.285254,0.022165,10.163564,0.225278,797406.274895,5.829286e+06,10115



Filtered Berlin Purchase Data:


,postcode_id,year,Obs,Radius,Obs_own,Radius_own,Effect_nown,lprice_qm,lprice_qm_se,price_qm,price_qm_se,target_x,target_y,target_id
11458,10115,2007,1121360,10,139180,2.5,-0.065757,7.902137,0.026401,2870.0420,75.772621,797406.274895,5.829286e+06,10115
11459,10115,2008,1121360,10,139180,2.5,-0.065757,8.086363,0.018491,3450.6157,63.804043,797406.274895,5.829286e+06,10115
11460,10115,2009,1121360,10,139180,2.5,-0.065757,8.114145,0.011043,3547.8264,39.180210,797406.274895,5.829286e+06,10115
11461,10115,2010,1121360,10,139180,2.5,-0.065757,8.150985,0.052313,3680.9639,192.563430,797406.274895,5.829286e+06,10115
11462,10115,2011,1121360,10,139180,2.5,-0.065757,8.295838,0.015158,4254.7168,64.492180,797406.274895,5.829286e+06,10115


In [ ]:
# File paths for GeoJSON files of Postal Codes and Neighboorhoods
plz_geojson = 'data/plz.geojson'
bezirksgrenzen_geojson = 'data/bezirksgrenzen.geojson'

# Load GeoJSON files
plz_gdf = gpd.read_file(plz_geojson)
bezirks_gdf = gpd.read_file(bezirksgrenzen_geojson)


print("\nPostal Code GeoDataFrame:")
display(plz_gdf.head())

print("\nNeighborhood GeoDataFrame:")
display(bezirks_gdf.head())



Postal Code GeoDataFrame:


,plz,geometry
0,10115,"POLYGON ((13.36586 52.53566, 13.36829 52.53329..."
1,10117,"POLYGON ((13.37374 52.5278, 13.37382 52.5277, ..."
2,10119,"POLYGON ((13.39902 52.52701, 13.40134 52.52631..."
3,10178,"POLYGON ((13.39902 52.52701, 13.39877 52.52679..."
4,10179,"POLYGON ((13.40305 52.51217, 13.40261 52.51186..."



Neighborhood GeoDataFrame:


,gml_id,Gemeinde_name,Gemeinde_schluessel,Land_name,Land_schluessel,Schluessel_gesamt,geometry
0,s_wfs_alkis_bezirk.F176__1,Reinickendorf,012,Berlin,11,11000012,"MULTIPOLYGON (((13.32074 52.6266, 13.32045 52...."
1,s_wfs_alkis_bezirk.F176__2,Charlottenburg-Wilmersdorf,004,Berlin,11,11000004,"MULTIPOLYGON (((13.32111 52.52446, 13.32103 52..."
2,s_wfs_alkis_bezirk.F176__3,Treptow-Köpenick,009,Berlin,11,11000009,"MULTIPOLYGON (((13.57925 52.39083, 13.57958 52..."
3,s_wfs_alkis_bezirk.F176__4,Pankow,003,Berlin,11,11000003,"MULTIPOLYGON (((13.50481 52.6196, 13.50467 52...."
4,s_wfs_alkis_bezirk.F176__5,Neukölln,008,Berlin,11,11000008,"MULTIPOLYGON (((13.45832 52.48569, 13.45823 52..."


Merge Postal Codes and Neighboorhoods with Price Data

In [ ]:
# Load postcode-to-neighborhood mapping
postal_to_neighborhood_df = pd.read_csv('data/Postcode_BerlinDistricts.csv')

# Rename columns
postal_to_neighborhood_df.rename(columns={'District': 'neighborhood', 'Postcode': 'postcode_id'}, inplace=True)

# Convert postal code to string format
postal_to_neighborhood_df['postcode_id'] = postal_to_neighborhood_df['postcode_id'].astype(str)

# Merge with rental data
berlin_rent_data = berlin_rent_data.merge(postal_to_neighborhood_df[['postcode_id', 'neighborhood']], on='postcode_id', how='left')


display(berlin_rent_data[['postcode_id', 'neighborhood']].drop_duplicates())


,postcode_id,neighborhood
0,10115,Mitte
17,10117,Mitte
34,10119,Mitte
35,10119,Pankow
68,10178,Mitte
...,...,...
4029,14197,Charlottenburg-Wilmersdorf
4030,14197,Steglitz-Zehlendorf
4031,14197,Tempelhof-Schöneberg
4080,14199,Charlottenburg-Wilmersdorf


In [ ]:
# Rename columns in GeoDataFrame
plz_gdf.rename(columns={'plz': 'postcode_id'}, inplace=True)

# Check column names
print("PLZ GeoDataFrame Columns:", plz_gdf.columns)
print("Berlin Rent Data Columns:", berlin_rent_data.columns)

# Merge GeoDataFrame with rental data based on 'postcode_id'
merged_gdf = plz_gdf.merge(berlin_rent_data, left_on="postcode_id", right_on="postcode_id", how="left")

# Ensure merged_gdf is a GeoDataFrame and has spatial information
merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry')


print(merged_gdf.head())


PLZ GeoDataFrame Columns: Index(['postcode_id', 'geometry'], dtype='object')
Berlin Rent Data Columns: Index(['postcode_id', 'year', 'Obs', 'Radius', 'Obs_own', 'Radius_own',
       'Effect_nown', 'lprice_qm', 'lprice_qm_se', 'price_qm', 'price_qm_se',
       'target_x', 'target_y', 'target_id', 'neighborhood'],
      dtype='object')
  postcode_id                                           geometry    year  \
0       10115  POLYGON ((13.36586 52.53566, 13.36829 52.53329...  2007.0   
1       10115  POLYGON ((13.36586 52.53566, 13.36829 52.53329...  2008.0   
2       10115  POLYGON ((13.36586 52.53566, 13.36829 52.53329...  2009.0   
3       10115  POLYGON ((13.36586 52.53566, 13.36829 52.53329...  2010.0   
4       10115  POLYGON ((13.36586 52.53566, 13.36829 52.53329...  2011.0   

         Obs  Radius   Obs_own  Radius_own  Effect_nown  lprice_qm  \
0  1330366.0    10.0  193637.0         2.5     0.005153   2.028197   
1  1330366.0    10.0  193637.0         2.5     0.005153   2.089856 

In [ ]:
# Rename 'Gemeinde_name' to 'neighborhood'
bezirks_gdf.rename(columns={'Gemeinde_name': 'neighborhood'}, inplace=True)

# Merge with the rent data using 'neighborhood'
merged_gdf = bezirks_gdf.merge(berlin_rent_data, on="neighborhood", how="left")

# Ensure merged_gdf is a GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry')


print(merged_gdf.head())


                       gml_id   neighborhood Gemeinde_schluessel Land_name  \
0  s_wfs_alkis_bezirk.F176__1  Reinickendorf                 012    Berlin   
1  s_wfs_alkis_bezirk.F176__1  Reinickendorf                 012    Berlin   
2  s_wfs_alkis_bezirk.F176__1  Reinickendorf                 012    Berlin   
3  s_wfs_alkis_bezirk.F176__1  Reinickendorf                 012    Berlin   
4  s_wfs_alkis_bezirk.F176__1  Reinickendorf                 012    Berlin   

  Land_schluessel Schluessel_gesamt  \
0              11          11000012   
1              11          11000012   
2              11          11000012   
3              11          11000012   
4              11          11000012   

                                            geometry postcode_id  year  \
0  MULTIPOLYGON (((13.32074 52.6266, 13.32045 52....       13403  2007   
1  MULTIPOLYGON (((13.32074 52.6266, 13.32045 52....       13403  2008   
2  MULTIPOLYGON (((13.32074 52.6266, 13.32045 52....       13403  2009   


**Visualizations**

Maps, Histograms and Tables of rent and purchase prices by neighboorhoods and by year

In [ ]:
import ipywidgets as widgets
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import matplotlib.colors as mcolors
from matplotlib.colorbar import Colorbar

# Define year range for the slider based on the data
year_slider = widgets.IntSlider(
    min=merged_gdf['year'].min(),
    max=merged_gdf['year'].max(),
    step=1,
    value=merged_gdf['year'].min(),
    description="Year:"
)

visualization_selector = widgets.Dropdown(
    options=['Map', 'Histogram', 'Table'],
    value='Map',
    description="View:"
)


# MAP: Function to generate map of rental prices by neighborhood for a given yea
def plot_map(year):
    year_data = merged_gdf[merged_gdf['year'] == year]

    # Compute the colormap
    min_price, max_price = year_data['lprice_qm'].min(), year_data['lprice_qm'].max()
    norm = mcolors.Normalize(vmin=min_price, vmax=max_price)
    cmap = plt.get_cmap('RdPu')

    # Plot the neighborhoods with rental price per m²
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))

    # Plot the neighborhood polygons
    year_data.plot(column='lprice_qm', ax=ax, legend=False,
                   cmap=cmap, edgecolor='black', linewidth=0.8, norm=norm)

    # Clean up the map's appearance
    ax.set_title(f'Berlin Neighborhoods - Rental Prices per m² ({year})')
    ax.set_axis_off()  # Turn off the axis to clean up the display

    for obj in ax.get_children():
        if isinstance(obj, Colorbar):
            obj.remove()

    # Create a custom colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation="horizontal", fraction=0.02, pad=0.04)
    cbar.set_label("Rental Price per m² (€)", fontsize=10)

    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

    plt.show()


# HISTOGRAM: Function to plot histogram of rental prices by neighborhood for a given year
def plot_histogram(year):
    year_data = merged_gdf[merged_gdf['year'] == year]
    neighborhood_avg_price = year_data.groupby('neighborhood')['lprice_qm'].mean().sort_values()

    # Use the same colormap as the map
    cmap = plt.get_cmap('RdPu')
    norm = mcolors.Normalize(vmin=neighborhood_avg_price.min(), vmax=neighborhood_avg_price.max())
    colors = [cmap(norm(val)) for val in neighborhood_avg_price]

    # Plot histogram with matching color scale
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.barh(neighborhood_avg_price.index, neighborhood_avg_price.values, color=colors)
    ax.set_xlabel("Average Rental Price (€ per m²)")
    ax.set_title(f"Rental Prices by Neighborhood - {year}")

    plt.show()


# TABLE: Function to show table of average rental prices by neighborhood for a given year
def show_table(year):
    year_data = merged_gdf[merged_gdf['year'] == year]

    # Group by neighborhood and calculate the mean rental price for each neighborhood
    neighborhood_avg_price = year_data.groupby('neighborhood').agg({'lprice_qm': 'mean'}).sort_values(by="lprice_qm", ascending=False)

    # Display the table of average prices by neighborhood
    display(neighborhood_avg_price)


# Function to update visualization based on user input
def update_visualization(year, visualization_type):
    clear_output(wait=True)  # Clear previous output to prevent clutter

    # Plot based on user selection
    if visualization_type == "Map":
        plot_map(year)
    elif visualization_type == "Histogram":
        plot_histogram(year)
    elif visualization_type == "Table":
        show_table(year)

# Link widgets to the function using interactive
interactive_output = widgets.interactive(update_visualization, year=year_slider, visualization_type=visualization_selector)

# Display widgets only once
display(interactive_output)


interactive(children=(IntSlider(value=2007, description='Year:', max=2023, min=2007), Dropdown(description='Vi…

In [ ]:
import ipywidgets as widgets
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import matplotlib.colors as mcolors
from matplotlib.colorbar import Colorbar

# Define year range for the slider (based on your data)
year_slider = widgets.IntSlider(
    min=merged_gdf['year'].min(),
    max=merged_gdf['year'].max(),
    step=1,
    value=merged_gdf['year'].min(),
    description="Year:"
)

visualization_selector = widgets.Dropdown(
    options=['Map', 'Histogram', 'Table'],
    value='Map',
    description="View:"
)


# MAP: Function to generate map of purchase prices by neighborhood for a given year
def plot_map(year):
    year_data = merged_gdf[merged_gdf['year'] == year]

    # Compute the colormap and apply normalization for purchase prices
    min_price, max_price = year_data['price_qm'].min(), year_data['price_qm'].max()
    norm = mcolors.Normalize(vmin=min_price, vmax=max_price)
    cmap = plt.get_cmap('YlGnBu')

    # Plot the neighborhoods with purchase price per m²
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))

    # Plot the neighborhood polygons
    year_data.plot(column='price_qm', ax=ax, legend=False,
                   cmap=cmap, edgecolor='black', linewidth=0.8, norm=norm)

    # Clean up the map's appearance
    ax.set_title(f'Berlin Neighborhoods - Purchase Prices per m² ({year})')
    ax.set_axis_off()  # Turn off the axis to clean up the display

    for obj in ax.get_children():
        if isinstance(obj, Colorbar):  # Use the correct Colorbar reference
            obj.remove()

    # Create a custom colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Create a dummy array for the colorbar
    cbar = fig.colorbar(sm, ax=ax, orientation="horizontal", fraction=0.02, pad=0.04)
    cbar.set_label("Purchase Price per m² (€)", fontsize=10)

    # Ensure the map doesn't display any other artifacts
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

    plt.show()


# HISTOGRAM: Function to plot histogram of purchase prices by neighborhood for a given year
def plot_histogram(year):
    year_data = merged_gdf[merged_gdf['year'] == year]
    neighborhood_avg_price = year_data.groupby('neighborhood')['price_qm'].mean().sort_values()

    # Use the same colormap as the map
    cmap = plt.get_cmap('YlGnBu')
    norm = mcolors.Normalize(vmin=neighborhood_avg_price.min(), vmax=neighborhood_avg_price.max())
    colors = [cmap(norm(val)) for val in neighborhood_avg_price]

    # Plot histogram with matching yellow-green-blue color scale
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.barh(neighborhood_avg_price.index, neighborhood_avg_price.values, color=colors)
    ax.set_xlabel("Average Purchase Price (€ per m²)")
    ax.set_title(f"Purchase Prices by Neighborhood - {year}")

    plt.show()


# TABLE: Function to show table of average purchase prices by neighborhood for a given year
def show_table(year):
    year_data = merged_gdf[merged_gdf['year'] == year]

    # Group by neighborhood and calculate the mean purchase price for each neighborhood
    neighborhood_avg_price = year_data.groupby('neighborhood').agg({'price_qm': 'mean'}).sort_values(by="price_qm", ascending=False)

    # Display the table of average prices by neighborhood
    display(neighborhood_avg_price)


# Function to update visualization based on user input
def update_visualization(year, visualization_type):
    clear_output(wait=True)  # Clear previous output to prevent clutter

    # Plot based on user selection
    if visualization_type == "Map":
        plot_map(year)
    elif visualization_type == "Histogram":
        plot_histogram(year)
    elif visualization_type == "Table":
        show_table(year)

# Link widgets to the function using interactive
interactive_output = widgets.interactive(update_visualization, year=year_slider, visualization_type=visualization_selector)

# Display widgets only once
display(interactive_output)


interactive(children=(IntSlider(value=2007, description='Year:', max=2023, min=2007), Dropdown(description='Vi…

Line Graphs of rend and purchase price trends over the years by Neighboorhoods

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Dropdown widget for selecting neighborhood
neighborhood_selector = widgets.Dropdown(
    options=merged_gdf['neighborhood'].unique(),  # Get unique neighborhood names
    description='Neighborhood:',
    style={'description_width': 'initial'}
)

# Function to update the rental price plot based on selected neighborhood
def update_rent_plot(neighborhood):
    plt.figure(figsize=(10, 6))

    # Filter and group data for the selected neighborhood, calculate the average rental price per year
    neighborhood_data = merged_gdf[merged_gdf['neighborhood'] == neighborhood]
    avg_rent_price = neighborhood_data.groupby('year')['lprice_qm'].mean()

    # Plot the rental price trend for the selected neighborhood (average per year)
    # Use a similar color of the previous visualization for the line
    plt.plot(avg_rent_price.index, avg_rent_price.values, marker='o', label=f'Rental Price in {neighborhood}', color='#D10072')  # Pinkish magenta

    plt.title(f'Average Rental Prices in {neighborhood} by Year')
    plt.xlabel('Year')
    plt.ylabel('Rental Price (€ per m²)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Display the interactive rental price plot
widgets.interactive(update_rent_plot, neighborhood=neighborhood_selector)


interactive(children=(Dropdown(description='Neighborhood:', options=('Reinickendorf', 'Charlottenburg-Wilmersd…

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Dropdown widget for selecting neighborhood
neighborhood_selector_purchase = widgets.Dropdown(
    options=merged_gdf['neighborhood'].unique(),  # Get unique neighborhood names
    description='Neighborhood:',
    style={'description_width': 'initial'}
)

# Function to update the purchase price plot based on selected neighborhood
def update_purchase_plot(neighborhood):
    plt.figure(figsize=(10, 6))

    # Filter and group data for the selected neighborhood, calculate the average purchase price per year
    neighborhood_data = merged_gdf[merged_gdf['neighborhood'] == neighborhood]
    avg_purchase_price = neighborhood_data.groupby('year')['price_qm'].mean()

    # Plot the purchase price trend for the selected neighborhood (average per year)
    # Use a similar color of the previous visualization for the line
    plt.plot(avg_purchase_price.index, avg_purchase_price.values, marker='o', label=f'Purchase Price in {neighborhood}', color='#2D8B8A')  # Bluish green

    plt.title(f'Average Purchase Prices in {neighborhood} by Year')
    plt.xlabel('Year')
    plt.ylabel('Purchase Price (€ per m²)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Display the interactive purchase price plot
widgets.interactive(update_purchase_plot, neighborhood=neighborhood_selector_purchase)


interactive(children=(Dropdown(description='Neighborhood:', options=('Reinickendorf', 'Charlottenburg-Wilmersd…